In [2]:
%matplotlib inline
%load_ext autoreload

%autoreload 2

# pandas 모듈은 흔히 pd라는 약칭으로 사용된다. Global 표준에 가깝다.
import pandas as pd
import time
from IPython.display import HTML
from IPython.display import display

# 커스텀 라이브러리를 import한다.
from pbp_plot import *
from misc import *

# precision 세팅을 한다. 내부 값은 소수점 5자리까지 표시되도록 저장하고, display 시에는 1자리로 표시하도록 한다.
pd.set_option('precision', 5)
pd.set_option('display.float_format', lambda x: '%.1f' % x)
#set_fonts()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import datetime
import requests
from bs4 import BeautifulSoup
import time
from dateutil.relativedelta import relativedelta

In [89]:
def get_game_ids(start_date, end_date):
    timetable_url = 'https://sports.news.naver.com/'\
                    'kbaseball/schedule/index.nhn?month='
    
    mon1 = start_date.replace(day=1)
    r = []
    while mon1 <= end_date:
        r.append(mon1)
        mon1 += relativedelta(months=1)
    
    game_ids = []
    
    for d in r:
        month = d.month
        year = d.year
        
        sch_url = timetable_url + f'{month}&year={year}'

        response = requests.get(sch_url)
        soup = BeautifulSoup(response.text, 'lxml')
        response.close()

        buttons = soup.findAll('span',
                               attrs={'class': 'td_btn'})

        for btn in buttons:
            gid = btn.a['href'].split('gameId=')[1]
            gid_date = datetime.date(int(gid[:4]),
                                     int(gid[4:6]),
                                     int(gid[6:8]))
            if start_date <= gid_date <= end_date:
                game_ids.append(gid)
    return game_ids

In [101]:
start_date=datetime.date(2018, 3, 21)
end_date=datetime.date(2018, 4, 2)
start_time = time.time()
game_ids = get_game_ids(start_date, end_date)
print(f'elapsed time: {(time.time() - start_time):.2f} sec')
print(f'len(gids): {len(game_ids)}')

elapsed time: 0.79 sec
len(gids): 41


In [108]:
from tqdm import tqdm, tqdm_notebook, trange
import json

20180717LGWO02018

In [212]:
start_date=datetime.date(2018, 7, 17)
end_date=datetime.date(2018, 7, 17)

start_time1 = time.time()
#########################################
relay_url = 'http://m.sports.naver.com/ajax/baseball/'\
            'gamecenter/kbo/relayText.nhn'
record_url = 'http://m.sports.naver.com/ajax/baseball/'\
            'gamecenter/kbo/record.nhn'

now = datetime.datetime.now().date()

game_ids = get_game_ids(start_date, end_date)
end_time1 = time.time()

skipped = 0
done = 0
start_time2 = time.time()
for gid in tqdm([game_ids[1]]):
    gid_to_date = datetime.date(int(gid[:4]),
                                int(gid[4:6]),
                                int(gid[6:8]))
    if gid_to_date > now:
        skipped += 1
        continue
    params = {
                'gameId': gid,
                'half': '1'
            }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
                      'AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/59.0.3071.115 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        'Host': 'm.sports.naver.com',
        'Referer': 'http://m.sports.naver.com/baseball/gamecenter/kbo/index.nhn?&gameId='
                   + gid
                   + '&tab=relay'
    }
    
    response = requests.get(relay_url,
                            params=params,
                            headers=headers)
    if response is None:
        skipped += 1
        continue
    
    js = response.json()
    if isinstance(js, str):
        js = json.loads(js)
    response.close()
    last_inning = js['currentInning']

    if last_inning is None:
        skipped += 1
        continue

    txt = {}
    txt['relayList'] = []
    for x in js['relayList']:
        txt['relayList'].append(x)

    txt['homeTeamLineUp'] = js['homeTeamLineUp']
    txt['awayTeamLineUp'] = js['awayTeamLineUp']

    txt['stadium'] = js['schedule']['stadium']
    
    for inn in range(2, last_inning + 1):
        params = {
            'gameId': gid,
            'half': str(inn)
        }

        response = requests.get(relay_url, params=params, headers=headers)
        if response is not None:
            js = response.json()
            if isinstance(js, str):
                js = json.loads(js)
            response.close()
            for x in js['relayList']:
                txt['relayList'].append(x)
        else:
            skipped += 1
            response.close()
            break
end_time2 = time.time()

print(f'schedule get time: {(end_time1 - start_time1):.2f} sec')
print(f'get json time: {(end_time2 - start_time2):.2f} sec')

100%|██████████| 1/1 [00:00<00:00,  1.13it/s]

schedule get time: 0.30 sec
get json time: 0.89 sec


In [214]:
rl = txt['relayList']